In [ ]:
#Section 2.1
#Advanced Content-Based Recommendation :

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

ratings = pd.read_csv("/content/sample_data/rating.csv")

metadata = pd.read_csv("/content/sample_data/anime.csv")


#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct a reverse map of indices and movie id
indices = pd.Series(metadata.index, index=metadata['anime_id']).drop_duplicates()

name_dic = dict(zip(list(range(N)), (metadata["name"])))
id_dic = dict(zip(list(range(N)), (metadata["anime_id"])))

# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# Apply clean_data function to your features.
features = ['name', 'genre', 'type', 'rating']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)


def create_soup(x):
    return ' ' + x['name'] + ' ' + x['genre'] + ' ' + x['type'] + ' ' + x['rating']

# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)


# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)


# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['anime_id']).drop_duplicates()
indices_na = pd.Series(metadata.index, index=metadata['name']).drop_duplicates()

# get top videos that user has watched
def get_movies(user_id):

  select = ratings.loc[ratings['user_id'] == user_id]
  sel = select[['anime_id', 'rating']]
  sel_sorted = sel.sort_values(by = 'rating', ascending = False)

  sel_mov = sel_sorted['anime_id'].tolist()

  all_mov = sel_sorted['anime_id'].tolist()

  m_top = sel_mov[0:10]

  return m_top , all_mov
  

N = len(metadata['name'])
	

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(top_movies, cosine_sim=cosine_sim2):

  rec_mov = []

  for i in top_movies:

    # Get the index of the movie that matches the title
    idx = indices[i]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the most similar movies
    sim_scores = sim_scores[1:3]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    for j in movie_indices:
      vid_id = id_dic[j]
      vid_name = name_dic[j]
      if (vid_name not in rec_mov) and (vid_id not in all_mov):
        rec_mov.append(vid_name)


 # Return the top most similar movies
  return rec_mov


print("")
print("for user id 3")
print("")

print("Suggested videos :")

top_movies , all_mov = get_movies(3)

re = get_recommendations(top_movies , cosine_sim2)[0:10]

print("")

for i in re:

 print(i)
 print("")
